In [2]:
import pkg_resources
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from psifr import fr
import psifr.transitions as tr
import importlib
data_file = pkg_resources.resource_filename('psifr', 'data/Morton2013.csv')

In [3]:
df = pd.read_csv(data_file, dtype={'category': 'category'})
df.category.cat.as_ordered(inplace=True)

study = df.query('trial_type == "study"').copy()
recall = df.query('trial_type == "recall"').copy()
data = fr.merge_lists(study, recall, 
                      list_keys=['list_type', 'list_category'], 
                      study_keys=['category'])

In [4]:
list1 = data.loc[(data['subject'] == 1) & (data['list'] == 37)]
rec1 = list1.sort_values('output')
rec1

,subject,list,item,input,output,recalled,repeat,intrusion,list_type,list_category,category
903,1,37,WATCH,24.0,1.0,True,0,False,pure,obj,obj
901,1,37,SKATEBOARD,22.0,2.0,True,0,False,pure,obj,obj
897,1,37,KNIFE,18.0,3.0,True,0,False,pure,obj,obj
896,1,37,TROPHY,17.0,4.0,True,0,False,pure,obj,obj
902,1,37,VASE,23.0,5.0,True,0,False,pure,obj,obj
889,1,37,DRESSER,10.0,6.0,True,0,False,pure,obj,obj
880,1,37,LAWN MOWER,1.0,7.0,True,0,False,pure,obj,obj
898,1,37,INTERCOM,19.0,8.0,True,0,False,pure,obj,obj
890,1,37,MODEM,11.0,9.0,True,0,False,pure,obj,obj
894,1,37,BARCODE SCANNER,15.0,10.0,True,0,False,pure,obj,obj


In [64]:
pool_items = list(range(1, 25))
recall_items = rec1.query('recalled')['input'].to_list()
pool_category = rec1.query('repeat == 0 and ~intrusion').sort_values('input')['category'].to_list()
recall_category = rec1.query('recalled')['category'].to_list()

In [75]:
importlib.reload(tr)
actual, possible = tr.count_lags(pool_items, [recall_items],
                                 [pool_category], [recall_category],
                                 lambda x, y: x == y)

In [77]:
possible

-23     1
-22     2
-21     3
-20     3
-19     3
-18     4
-17     6
-16     7
-15     7
-14     8
-13     8
-12     9
-11     9
-10     8
-9      8
-8     11
-7     11
-6      9
-5      8
-4     10
-3     11
-2     10
-1      7
 0      0
 1      9
 2      7
 3      5
 4      5
 5      7
 6      5
 7      2
 8      1
 9      3
 10     3
 11     2
 12     1
 13     2
 14     2
 15     1
 16     0
 17     0
 18     1
 19     1
 20     1
 21     0
 22     0
dtype: int64

In [55]:
importlib.reload(tr)
masker = tr.transitions_masker(pool_items, recall_items, 
                               pool_items, recall_items)
steps = [[int(x), int(y), z.tolist()] for x, y, z in masker]
print(recall_items)
print()
for step in steps:
    print(step)

[24.0, 22.0, 18.0, 17.0, 23.0, 10.0, 1.0, 19.0, 11.0, 15.0, 7.0, 16.0, 20.0, 12.0, nan, 24.0]

[24, 22, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]
[22, 18, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23]]
[18, 17, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23]]
[17, 23, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 23]]
[23, 10, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21]]
[10, 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21]]
[1, 19, [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 19, 20, 21]]
[19, 11, [2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 20, 21]]
[11, 15, [2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 20, 21]]
[15, 7, [2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 16, 20, 21]]
[7, 16, [2, 3, 4, 5, 6, 8, 9, 12, 13, 14, 16, 20, 21]]
[16, 20, [2, 3, 4, 5, 6, 8, 9, 12, 13, 14, 20, 21]]
[20, 12, [2, 3, 4, 5, 6, 8, 9, 12, 13

In [26]:
def masker(pool, output, pool_test, output_test, test):
    n = 0
    possible = pool.copy()
    possible_test = pool_test.copy()
    while n < len(output) - 1:
        # test if the previous item is in the pool
        if output[n] not in possible:
            n += 1
            continue

        # remove the item from the pool
        ind = possible.index(output[n])
        del possible[ind]
        del possible_values[ind]

        # test if the current item is in the pool
        if output[n + 1] not in pool:
            n += 1
            continue

        # test if this transition is included
        if not test(output_values[n], output_values[n + 1]):
            n += 1
            continue

        # get included possible items
        poss = np.array(possible)
        poss = poss[test(output_values[n], np.array(possible_values))]
        n += 1
        yield output[n], output[n + 1], poss

In [33]:
pool = list(range(1, 9))
pool_values = [1, 1, 1, 1, 2, 2, 2, 2]
output = np.array([1, 3, 4, 8, 5, 4, 6])
output_values = np.array([1, 1, 1, 2, 2, 1, 2])
#test = lambda x, y: np.ones_like(y, dtype=bool)
test = lambda x, y: x == y

In [34]:
for prev, curr, poss in masker(pool, output, pool_values, output_values, test):
    print(prev, curr, poss)

1 3 [2 3 4]
3 4 [2 4]
8 5 [5 6 7]


In [43]:
pool_values = list(range(1, 9))
pool = [1, 1, 1, 1, 2, 2, 2, 2]
output_values = np.array([1, 3, 4, 8, 5, 4, 6])
output = np.array([1, 1, 1, 2, 2, 1, 2])
#test = lambda x, y: np.ones_like(y, dtype=bool)
test = lambda x, y: (x <= 4) & (y <= 4)

In [44]:
for prev, curr, poss in masker(pool, output, pool_values, output_values, test):
    print(prev, curr, poss)

1 1 [1 1 1]
1 1 [1 1]
